#40 Slice

In [41]:
#in_path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_files/labels"

#out_path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_groups/labels"

in_path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_files/images"
out_path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_groups/images"

In [ ]:
!pip install glob2

to moving the files

In [ ]:
!pip install pytest-shutil                    

In [30]:
from glob import glob
import shutil
import os

In [37]:

def move_files(path, count):
    for i in glob(path + "/*"):
        i_name = os.path.basename(os.path.normpath(i))
        num_of_folders = int(len(glob(i+"/*"))/count)

        for j in range(num_of_folders):
            out_new = os.path.join(out_path, i_name+"_"+str(j))
            os.mkdir(out_new)

            for k, file in enumerate(glob(i+"/*")):
                if k ==40+1:
                    break
                shutil.move(file, out_new)

In [50]:

def hello(path_in, path_out, count):
    for i in glob(path + "/*"):
        i_name = os.path.basename(os.path.normpath(i))
        num_of_folders = int(len(glob(i+"/*"))/count)
        print(len(glob(i+"/*")))
        print(num_of_folders)

        for j in range(num_of_folders):
            out_new = os.path.join(path_out, i_name+"_"+str(j))
            os.mkdir(out_new)

            for k, file in enumerate(glob(i+"/*")):
                if k ==count+1:
                    break
                shutil.move(file, out_new)

90
2
40
1


In [77]:
path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_files/images"

k = glob(path+"/*")[0]

size_count = []
for i in glob(path+"/*"):
    size_count.append(len(glob(i+"/*")))

In [78]:
min(size_co)

[90, 40]